In [ ]:
import rasterio

vv_path = "/home/stagiaire/Téléchargements/Browser_images/2019-03-30-00:00_2019-03-30-23:59_Sentinel-1_IW_VV+VH_VV_(Raw).tiff"
vh_path = "/home/stagiaire/Téléchargements/Browser_images/2019-03-30-00:00_2019-03-30-23:59_Sentinel-1_IW_VV+VH_VH_(Raw).tiff"

with rasterio.open(vv_path) as src:
    # Display the number of bands
    band_count = src.count
    print("Number of bands:", band_count)

    # Display the resolution
    resolution = src.res
    print("Resolution:", resolution)
    
    VV = src.read(1)
    
with rasterio.open(vh_path) as src:
    # Display the number of bands
    band_count = src.count
    print("Number of bands:", band_count)

    # Display the resolution
    resolution = src.res
    print("Resolution:", resolution)
    
    VH = src.read(1)

from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS

file_paths = [vv_path, vh_path]

# Function to project the image into a specific coordinate system
def project_image(input_path, output_path, output_crs):
    with rasterio.open(input_path) as src:
        # Calculate the default transform
        transform, width, height = calculate_default_transform(src.crs, output_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': output_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Reproject the image and save the result
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=output_crs,
                    resampling=Resampling.nearest)

# Project each image into the EPSG:32723 coordinate system (UTM Zone 27S)
for path in file_paths:
    output_path = f"{path.split('.')[0]}_proj.tiff"
    project_image(path, output_path, CRS.from_epsg(32723))

# Function to obtain the spatial resolution of a TIFF file
def get_resolution(file_path):
    with rasterio.open(file_path) as src:
        resolution_x = src.res[0]
        resolution_y = src.res[1]
    return resolution_x, resolution_y

# For each file, get and display the spatial resolution
for path in file_paths:
    proj_path = f"{path.split('.')[0]}_proj.tiff"
    resolution_x, resolution_y = get_resolution(proj_path)
    print(f"Spatial resolution of {proj_path.split('.')[0].split('/')[-1]}: {resolution_x}m x {resolution_y}m")

def resample_image(input_path, output_path, resolution_x, resolution_y):
    with rasterio.open(input_path) as src:
        # Calculate the default transform
        transform, width, height = calculate_default_transform(src.crs, src.crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'transform': transform,
            'width': width,
            'height': height,
            'dtype': 'float32'
        })

        # Create a new image with the specified resolution
        kwargs['transform'] = rasterio.Affine(resolution_x, 0.0, src.bounds.left, 0.0, -resolution_y, src.bounds.top)
        kwargs['width'] = int((src.bounds.right - src.bounds.left) / resolution_x)
        kwargs['height'] = int((src.bounds.top - src.bounds.bottom) / resolution_y)

        # Resample the image and save the result
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=kwargs['transform'],
                    dst_crs=src.crs,
                    resampling=Resampling.bilinear)  # Bilinear resampling

# Resample each band to a resolution of 10 meters
for path in file_paths:
    proj_path = f"{path.split('.')[0]}_proj.tiff"
    output_path = f"{proj_path.split('.')[0]}_10m.tiff"
    resample_image(proj_path, output_path, 10, 10)
    